In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.amp import GradScaler, autocast
import pickle, os
from tqdm import tqdm

In [2]:
# Set up CIFAR-10 Dataset with Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='../data/train', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='../data/test', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Define the CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Check if a GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize model, loss function, and optimizer
model = SimpleCNN().to(device)  # Move model to the device
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


In [4]:

scaler = GradScaler()  # No need to specify device here

num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    # Use tqdm for progress bar
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch", leave=False)
    
    # Iterate over the data
    for i, (inputs, labels) in enumerate(progress_bar, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Enable autocasting for the forward pass
        with autocast(device_type='cuda'):  # Specify the device type correctly
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        # Scales the loss, and calls backward()
        scaler.scale(loss).backward()
        
        # scaler.step() unscales the gradients and calls or skips optimizer.step()
        scaler.step(optimizer)
        
        # Updates the scale for next iteration
        scaler.update()
        
        # define a new loss for the next iteration
        running_loss += loss.item()
        progress_bar.set_postfix(loss=f"{running_loss / (i + 1):.4f}")
    
    # close progress bar
    progress_bar.close()
    
print("Finished Training")


Finished Training


In [5]:
# Evaluate the Model on Test Data
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_loader:
        # Move inputs and labels to the device
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)  # Calculate loss
        test_loss += loss.item()

        # Get predicted class
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average test loss and accuracy
average_test_loss = test_loss / len(test_loader)
accuracy = 100 * correct / total

print(f"Test Loss: {average_test_loss:.3f}")
print(f"Test Accuracy: {accuracy:.2f}%")


Test Loss: 5.055
Test Accuracy: 69.15%


In [ ]:
# 4. Export the Model and Save Validation Data
# Save the trained model
torch.save(model.state_dict(), '../data/simple_cnn_fp16.pth')
print("Model saved as 'simple_cnn_fp16.pth'")

Model saved as 'simple_cnn_fp16.pth'


In [7]:
# Save the validation data to a file for later inference benchmarking
validation_data = []
for images, labels in test_loader:
    validation_data.append((images, labels))

with open('../data/validation_data.pkl', 'wb') as f:
    pickle.dump(validation_data, f)

print("Validation data saved as 'validation_data.pkl'")

Validation data saved as 'validation_data.pkl'
